<a href="https://colab.research.google.com/github/Harshkotkar/AI-agents-/blob/main/AI_Customer_Support_Assistant_for_E_commerce_(Myntra_Demo).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install -U langchain langchain_core langchain_community langchain-text-splitters
!pip install pypdf
!pip install langchain_google_genai
!pip install bs4


In [5]:

!pip install -U langchain_community

  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [10]:
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.9/745.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.9/280.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


In [4]:
# gemini setup
import getpass
import os
os.environ["GOOGLE_API_KEY"]=getpass.getpass("Enter the api key ")

Enter the api key ··········


In [12]:
import getpass
import os

pinecone_api_key = getpass.getpass("Enter your Pinecone API key: ")
pc = Pinecone(api_key=pinecone_api_key)

Enter your Pinecone API key: ··········


In [18]:
import bs4
from langchain_core.prompts import ChatPromptTemplate , PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from pinecone import Pinecone, ServerlessSpec
from pydantic import BaseModel, Field
from typing import Optional



In [9]:
llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash",temperature=0)


In [13]:
# embeddings = GoogleGenerativeAIEmbeddings(
#     model="models/text-embedding-004",
#     task_type="retrieval_document",
#     output_dimensionality=512
# )

#huggingface embeding for the pinecone
# index_name = "bge-base-index"

# if not pc.has_index(index_name):
#     pc.create_index(
#         name=index_name,
#         dimension=768,   # bge-base-en-v1.5 outputs 768-dim vectors
#         metric="cosine",
#         spec=ServerlessSpec(cloud="aws", region="us-east-1")
#     )


In [14]:
#google embeding modle for the pinecone
index_name = "gemini-index" # Rename to reflect the new model

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=768,   # Updated for text-embedding-004
        metric="cosine", # Gemini models work best with cosine or dot_product
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

In [20]:
#pydantic initalization
class CustomerQuery(BaseModel):
    query: str = Field(
        ...,
        min_length=5,
        description="Customer question"
    )
    user_id: Optional[str] = Field(
        None,
        description="Unique user identifier"
    )
    order_id: Optional[str] = Field(
        None,
        description="Order ID if query is order related"
    )


In [15]:
# Initialize Gemini Embedder
embedder = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004",
    task_type="retrieval_document" # Best for indexing/upserting
)

In [ ]:

# vector = embedder.embed_query("Pinecone is a vector database")
# index = pc.Index(index_name)
# index.upsert(vectors=[("1", vector, {"text": "Pinecone is a vector database"})])


In [ ]:
loder=WebBaseLoader(web_path="https://www.hri.org.au/health/learn/your-body/the-human-heart",
                    bs_kwargs=dict(
                        parse_only=bs4.SoupStrainer(id="main-content")
                    ))

##Output Validation Pydantic


In [ ]:

class SupportResponse(BaseModel):
    answer: str = Field(
        ...,
        description="Final AI-generated answer"
    )
    confidence: float = Field(
        ...,
        ge=0.0,
        le=1.0,
        description="Confidence score of the answer"
    )
    needs_human: bool = Field(
        ...,
        description="Should escalate to human support?"
    )
    source: str = Field(
        ...,
        description="dense | sparse | hybrid | web"
    )
